In [1]:
import pandas as pd
import numpy as np
import os 
import time

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

from multiprocessing import Pool

def data_generate(data_type_filled):
    time_in = 12
    time_out = 6
    num_sample = data_type_filled['PM2.5'].shape[0] -time_in -time_out
    num_loc = data_type_filled['PM2.5'].shape[1] -3
    num_feature = len(data_type_filled)
    data = []
    label = []
    data_filled_array = {}
    for type_now in data_type_filled:
        data_filled_array[type_now] = data_type_filled[type_now].iloc[:,3:].values
        
    start = time.time()
    for i in range(0,num_sample):
        data_now = np.zeros([time_in, num_feature, num_loc])
        label_now = np.zeros([time_out,num_loc])
        feature_now = -1
        
        # 判断date是否连续
        if data_type_filled['PM2.5'].iloc[i+time_in+time_out-1,0] - data_type_filled['PM2.5'].iloc[i,0] < 2:
            # 判断hour是否连续
            if data_type_filled['PM2.5'].iloc[i+time_in+time_out-1,1] == (data_type_filled['PM2.5'].iloc[i,1] + time_in+time_out-1)%24: 
                for type_now in data_type_filled:
                    feature_now = feature_now +1
                    data_now[:,feature_now,:] = data_filled_array[type_now][i:i+time_in,:]
                    if type_now == 'PM2.5':
                        label_now = data_filled_array[type_now][i+time_in:i+time_in+time_out,:]
                        
                data_now = data_now.reshape((time_in*num_feature,num_loc))
                data.append(data_now)
                label.append(label_now)
        
    end = time.time()
#     print('data generating lasted: '+str(end-start))
                
    data = np.array(data)
    label = np.array(label)
    return data, label

def Evaluation(label, predict):
    MAE = np.mean(np.abs(label - predict))
    RMSE = np.power(np.mean(np.power(label - predict,2)) ,0.5)

    label_grade = label.copy()
    label_grade[label_grade < 35] = 1
    label_grade[label_grade > 250] = 6
    label_grade[label_grade > 150] = 5
    label_grade[label_grade > 115] = 4
    label_grade[label_grade > 75] = 3
    label_grade[label_grade > 35] = 2
    
    predict_grade = predict.copy()
    predict_grade[predict_grade < 35] = 1
    predict_grade[predict_grade > 250] = 6
    predict_grade[predict_grade > 150] = 5
    predict_grade[predict_grade > 115] = 4
    predict_grade[predict_grade > 75] = 3
    predict_grade[predict_grade > 35] = 2
    
    res = np.zeros(label_grade.shape)
    res[label_grade == predict_grade] = 1
    num_cor = res.sum()
    num_all = res.shape[0] * res.shape[1]
    prec = num_cor/num_all
    return MAE,RMSE,prec

In [10]:
data_all = pd.read_csv("data_all.csv")
# data_extra = pd.read_csv("data_extra.csv")

In [14]:
# 删去无用的前一列
data_all = data_all.iloc[:,1:]
# data_extra = data_extra.iloc[:,3:]

# 删去缺失了一半PM2.5数据的植物园
data_all = data_all.drop(columns=['植物园'])

# # 删去20170702-20170708
# data_all = data_all.drop(index= range(148393,148456))



In [12]:
# 选取PM2.5数据
data_all = data_all[data_all['type']=='PM2.5']

In [52]:
data_all['密云水库'].iloc[0]

nan

In [56]:
np.isnan(data_all['密云水库'].iloc[1]) and np.isnan(data_all['密云水库'].iloc[0])

False

In [50]:
if data_all['密云水库'].iloc[0]:
    print('1')

1


In [57]:
count_allnan = 0
count_nan = 0
for col in range(3,data_all.shape[1]):
    for row in range(1,data_all.shape[0]-1):
        if np.isnan(data_all.iloc[row,col]):
            count_nan = count_nan + 1
            if np.isnan(data_all.iloc[row-1,col]) and np.isnan(data_all.iloc[row+1,col]):
                count_allnan = count_allnan+1
                

In [58]:
count_nan

73862

In [59]:
count_allnan

43996

In [60]:
# 查看全体数据缺失情况
na_count = data_all.isnull().sum().sort_values(ascending=False)
na_rate = na_count / len(data_all)
na_data = pd.concat([na_count,na_rate],axis=1,keys=['count','ratio'])
na_data

,count,ratio
南三环,4894,0.090382
琉璃河,4727,0.087298
东四环,4265,0.078766
八达岭,4038,0.074573
榆垡,3692,0.068183
通州,3085,0.056973
前门,2988,0.055182
东高村,2701,0.049882
密云水库,2590,0.047832
西直门北,2474,0.045690


In [16]:
# 划分数据集
data_train = data_all[data_all['date'] < 20200000]
data_train = data_train[data_train['date'] > 20150000]
data_val = data_all[data_all['date'] > 20200000]
data_test = data_all[data_all['date'] < 20150000]

In [17]:
# data_all_type = {}
# for data_type in data_all['type'][0:5]:
#     data_all_type[data_type] = data_all[data_all['type']==data_type]
# for data_type in data_extra['type'][0:7]:
#     data_all_type[data_type] = data_extra[data_extra['type']==data_type]

data_train_type = {}
for data_type in data_train['type'][0:5]:
    data_train_type[data_type] = data_train[data_train['type']==data_type]
data_val_type = {}
for data_type in data_val['type'][0:5]:
    data_val_type[data_type] = data_val[data_val['type']==data_type]
data_test_type = {}
for data_type in data_test['type'][0:5]:
    data_test_type[data_type] = data_test[data_test['type']==data_type]

In [18]:
# for type_now in data_all_type:
#     data_all_type[type_now] = data_all_type[type_now].fillna(method='ffill')
#     data_all_type[type_now] = data_all_type[type_now].fillna(method='bfill')
for type_now in data_train_type:
    data_train_type[type_now] = data_train_type[type_now].fillna(method='ffill')
    data_train_type[type_now] = data_train_type[type_now].fillna(method='bfill')
for type_now in data_val_type:
    data_val_type[type_now] = data_val_type[type_now].fillna(method='ffill')
    data_val_type[type_now] = data_val_type[type_now].fillna(method='bfill')
for type_now in data_test_type:
    data_test_type[type_now] = data_test_type[type_now].fillna(method='ffill')
    data_test_type[type_now] = data_test_type[type_now].fillna(method='bfill')

In [22]:
# 查看数据缺失情况
print('train')
for type_now in data_train_type:
    df = data_train_type[type_now] 
    na_count = df.isnull().sum().sort_values(ascending=False)
    na_rate = na_count / len(df)
    na_data = pd.concat([na_count,na_rate],axis=1,keys=['count','ratio'])
    print(na_data['count'].max())
print('val')
for type_now in data_val_type:
    df = data_val_type[type_now] 
    na_count = df.isnull().sum().sort_values(ascending=False)
    na_rate = na_count / len(df)
    na_data = pd.concat([na_count,na_rate],axis=1,keys=['count','ratio'])
    print(na_data['count'].max())
print('test')
for type_now in data_test_type:
    df = data_test_type[type_now] 
    na_count = df.isnull().sum().sort_values(ascending=False)
    na_rate = na_count / len(df)
    na_data = pd.concat([na_count,na_rate],axis=1,keys=['count','ratio'])
    print(na_data['count'].max())

train
0
val
0
test
0


In [19]:
# 制作时序数据集
train_data,train_label = data_generate(data_train_type)
val_data,val_label = data_generate(data_val_type)
test_data,test_label = data_generate(data_test_type)

In [20]:
train_data.shape

(39160, 12, 34)

In [9]:
train_data.shape

(42539, 12, 34)

In [21]:
def para_pass(args):
    return model_fitting(*args)

def model_fitting(model_name,train_data,train_label,ti,lo):
    if model_name =='DT':
        model = DecisionTreeRegressor(max_depth = 50)
    elif model_name =='RF':
        model = RandomForestRegressor(n_estimators=50,max_depth = 50)
    elif model_name =='Ada_DT':
        model = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(max_depth=50),n_estimators=50)
    elif model_name =='GBDT':
        model = GradientBoostingRegressor(n_estimators=10,max_depth = 50)
        
    model.fit(train_data[:,:,lo], train_label[:,ti,lo])
    return [model, ti, lo]

In [17]:
train_data

array([[[ 3. ,  6.9,  3. , ..., 33. ,  3. ,  3.4],
        [ 3. ,  3.4,  5.2, ..., 55. ,  3. ,  4. ],
        [ 3.9,  3. ,  4.4, ..., 41.1,  3. ,  5.5],
        ...,
        [ 7. ,  8.2,  3. , ..., 28.4,  3. ,  3.2],
        [ 6.6, 12.2,  5.1, ..., 39.2, 10.1,  7.3],
        [18.5, 14.4,  4.9, ..., 32.3, 29.7,  7.7]],

       [[ 3. ,  3.4,  5.2, ..., 55. ,  3. ,  4. ],
        [ 3.9,  3. ,  4.4, ..., 41.1,  3. ,  5.5],
        [ 4.2,  4.5,  3. , ..., 17.2,  3. ,  4. ],
        ...,
        [ 6.6, 12.2,  5.1, ..., 39.2, 10.1,  7.3],
        [18.5, 14.4,  4.9, ..., 32.3, 29.7,  7.7],
        [50.5, 35.5, 21.8, ..., 37.6, 46.2, 12.8]],

       [[ 3.9,  3. ,  4.4, ..., 41.1,  3. ,  5.5],
        [ 4.2,  4.5,  3. , ..., 17.2,  3. ,  4. ],
        [ 4.4,  4.4,  4. , ...,  9.9,  4.3,  4.7],
        ...,
        [18.5, 14.4,  4.9, ..., 32.3, 29.7,  7.7],
        [50.5, 35.5, 21.8, ..., 37.6, 46.2, 12.8],
        [57. , 32.7, 41.4, ..., 45.3, 44.3, 11.5]],

       ...,

       [[ 8. ,  8. ,  8.

In [23]:
# 训练并测试模型
print('training')
# DT = DecisionTreeRegressor(max_depth = 50)
# RF = RandomForestRegressor(n_estimators=10,max_depth = 50)
# Ada_DT = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(max_depth=50),n_estimators=10)
# GBDT = GradientBoostingRegressor(n_estimators=10,max_depth = 50)
    
models = ['DT']
train_predict_all = []
val_predict_all = []
test_predict_all = []

MultiNum = 16
pool = Pool(processes=MultiNum)

for model_now in models:
    print(model_now)
    start = time.time()
    train_predict = np.zeros(train_label.shape)
    val_predict = np.zeros(val_label.shape)
    test_predict = np.zeros(test_label.shape)
            
### 并行算法 ###    
    num_time = train_label.shape[1]
    num_loc = train_label.shape[2]
    
    para_model = [model_now]*num_loc*num_time
    para_train_data = [train_data]*num_loc*num_time
    para_train_label = [train_label]*num_loc*num_time
    para_loc = [lo for lo in range(num_loc)]*num_time

# 时间和空间都并行
    para_ti = [ti for ti in range(num_time) for lo in range(num_loc)]
    para_list = list(zip(para_model, para_train_data,para_train_label,para_ti,para_loc))
    model_fitted_list = pool.map(para_pass,para_list)
    
    for model_fitted, ti, lo in model_fitted_list:
        train_predict[:,ti,lo] = model_fitted.predict(train_data[:,:,lo])
        val_predict[:,ti,lo] = model_fitted.predict(val_data[:,:,lo])
        test_predict[:,ti,lo] = model_fitted.predict(test_data[:,:,lo])


# 时间串行，空间并行
#     for ti in range(num_time):
#         para_ti = [ti]*num_loc
#         para_list = list(zip(para_model, para_train_data,para_train_label,para_ti,para_loc))
#         model_fitted_list = pool.map(para_pass,para_list)
#         for lo in range(num_loc):
#             model_fitted = model_fitted_list[lo]
#             train_predict[:,ti,lo] = model_fitted.predict(train_data[:,:,lo])
#             val_predict[:,ti,lo] = model_fitted.predict(val_data[:,:,lo])
#             test_predict[:,ti,lo] = model_fitted.predict(test_data[:,:,lo])
            
    train_predict_all.append(train_predict)
    val_predict_all.append(val_predict)
    test_predict_all.append(test_predict)
    end = time.time()
    print('training lasted: '+str(end-start))
        
pool.close()
pool.join()
            

training
DT
training lasted: 26.324068784713745


In [24]:
# 模型并行性能 :GT 
for m in range(len(models)):
    print(models[m])
    print('train error')
    for i in range(6):
        MAE, RMSE,PREC= Evaluation(train_label[:,i,:], train_predict_all[m][:,i,:])
        print('time:'+str(i+1)+' '+'MAE = '+str(MAE)+' '+'RMSE = '+str(RMSE)+' '+'PREC = '+str(PREC))
    print('val error')
    for i in range(6):
        MAE, RMSE,PREC= Evaluation(val_label[:,i,:], val_predict_all[m][:,i,:])
        print('time:'+str(i+1)+' '+'MAE = '+str(MAE)+' '+'RMSE = '+str(RMSE)+' '+'PREC = '+str(PREC))
    print('test error') 
    for i in range(6):
        MAE, RMSE,PREC= Evaluation(test_label[:,i,:], test_predict_all[m][:,i,:])
        print('time:'+str(i+1)+' '+'MAE = '+str(MAE)+' '+'RMSE = '+str(RMSE)+' '+'PREC = '+str(PREC))      

DT
train error
time:1 MAE = 0.05235185806703889 RMSE = 2.1411068056503515 PREC = 0.9992406717538905
time:2 MAE = 0.10053102529014249 RMSE = 5.0924724218541755 PREC = 0.998771255182359
time:3 MAE = 0.14068874673842952 RMSE = 6.684687044899804 PREC = 0.9982665384846482
time:4 MAE = 0.1670474240983312 RMSE = 7.772946952804158 PREC = 0.9980472270624287
time:5 MAE = 0.19099556720868724 RMSE = 8.580953115142538 PREC = 0.9976664363396023
time:6 MAE = 0.21122054762371328 RMSE = 9.188646258301542 PREC = 0.9973419756053596
val error
time:1 MAE = 9.79873028030081 RMSE = 18.90146021493469 PREC = 0.8021541010770505
time:2 MAE = 15.395994039455172 RMSE = 28.310481456904668 PREC = 0.7121996685998343
time:3 MAE = 19.886002283488903 RMSE = 34.941206687292016 PREC = 0.6454121789560895
time:4 MAE = 23.45562622422438 RMSE = 39.93313305658166 PREC = 0.6017812758906379
time:5 MAE = 26.512668961854498 RMSE = 44.13321649842704 PREC = 0.5658864954432478
time:6 MAE = 29.20662636832948 RMSE = 48.1185961453789 PR

In [23]:
# 模型并行性能 :GBFT 10棵子树
for m in range(len(models)):
    print(models[m])
    print('train error')
    for i in range(6):
        MAE, RMSE,PREC= Evaluation(train_label[:,i,:], train_predict_all[m][:,i,:])
        print('time:'+str(i+1)+' '+'MAE = '+str(MAE)+' '+'RMSE = '+str(RMSE)+' '+'PREC = '+str(PREC))
    print('val error')
    for i in range(6):
        MAE, RMSE,PREC= Evaluation(val_label[:,i,:], val_predict_all[m][:,i,:])
        print('time:'+str(i+1)+' '+'MAE = '+str(MAE)+' '+'RMSE = '+str(RMSE)+' '+'PREC = '+str(PREC))
    print('test error') 
    for i in range(6):
        MAE, RMSE,PREC= Evaluation(test_label[:,i,:], test_predict_all[m][:,i,:])
        print('time:'+str(i+1)+' '+'MAE = '+str(MAE)+' '+'RMSE = '+str(RMSE)+' '+'PREC = '+str(PREC))      

GBDT
train error
time:1 MAE = 16.741882344222347 RMSE = 26.000435208563477 PREC = 0.70925918499702
time:2 MAE = 16.766887901311435 RMSE = 26.305504838801443 PREC = 0.7088242899595251
time:3 MAE = 16.78745389150949 RMSE = 26.567985033374878 PREC = 0.7085387388458757
time:4 MAE = 16.802985498345688 RMSE = 26.784782194675827 PREC = 0.7083437620564105
time:5 MAE = 16.8160327535023 RMSE = 26.962934975071818 PREC = 0.7081771329561938
time:6 MAE = 16.8273511515282 RMSE = 27.104494597574085 PREC = 0.7080616679780354
val error
time:1 MAE = 16.995216060475638 RMSE = 23.414267573223988 PREC = 0.6910069921943094
time:2 MAE = 19.792647422409395 RMSE = 28.300609027052857 PREC = 0.6419744717116349
time:3 MAE = 22.166639036771308 RMSE = 32.12044449781434 PREC = 0.6034980340506304
time:4 MAE = 24.26109295999262 RMSE = 35.20270439534844 PREC = 0.5692537139979469
time:5 MAE = 26.334541362026354 RMSE = 38.32470877313603 PREC = 0.5383892773441283
time:6 MAE = 27.941243748337428 RMSE = 40.997789555180375 PR

In [21]:
# 模型并行性能 :RF和Ada_DT都是50棵子树
for m in range(len(models)):
    print(models[m])
    print('train error')
    for i in range(6):
        MAE, RMSE,PREC= Evaluation(train_label[:,i,:], train_predict_all[m][:,i,:])
        print('time:'+str(i+1)+' '+'MAE = '+str(MAE)+' '+'RMSE = '+str(RMSE)+' '+'PREC = '+str(PREC))
    print('val error')
    for i in range(6):
        MAE, RMSE,PREC= Evaluation(val_label[:,i,:], val_predict_all[m][:,i,:])
        print('time:'+str(i+1)+' '+'MAE = '+str(MAE)+' '+'RMSE = '+str(RMSE)+' '+'PREC = '+str(PREC))
    print('test error') 
    for i in range(6):
        MAE, RMSE,PREC= Evaluation(test_label[:,i,:], test_predict_all[m][:,i,:])
        print('time:'+str(i+1)+' '+'MAE = '+str(MAE)+' '+'RMSE = '+str(RMSE)+' '+'PREC = '+str(PREC))      

RF
train error
time:1 MAE = 3.3675079199393516 RMSE = 8.24192910211966 PREC = 0.9326590270796488
time:2 MAE = 5.246291141826144 RMSE = 11.775614400181503 PREC = 0.8985325576668054
time:3 MAE = 6.701935247579758 RMSE = 14.23825125633195 PREC = 0.8721885660632527
time:4 MAE = 7.912974581541557 RMSE = 16.251429404911146 PREC = 0.850454185294325
time:5 MAE = 8.930252998906576 RMSE = 17.789144555571443 PREC = 0.8329422274093116
time:6 MAE = 9.80011363095991 RMSE = 19.078382868191643 PREC = 0.8173108967134657
val error
time:1 MAE = 6.432085196867393 RMSE = 12.463064900590798 PREC = 0.867109570202793
time:2 MAE = 10.143957268751905 RMSE = 18.679329110991905 PREC = 0.7956284258846772
time:3 MAE = 13.175705623861505 RMSE = 23.213265938212942 PREC = 0.7410563830405392
time:4 MAE = 15.690035096950883 RMSE = 26.55060614839297 PREC = 0.6990741637451819
time:5 MAE = 17.879254802536256 RMSE = 29.33959506776362 PREC = 0.6627573650467761
time:6 MAE = 19.740808420872987 RMSE = 31.753721109844978 PREC = 